In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import os


url = "https://dev.to/latest"
ua = UserAgent()
userAgent = ua.random
headers = {"user-agent": userAgent}
page = requests.get(url, headers = headers)

soup = BeautifulSoup(page.content, "html.parser")

In [6]:
blog_box = soup.find_all("div", class_ = "crayons-story__body")

links = []
titles = []
time_uploaded = []
authors = []
tags = []
reading_times = []

for box in blog_box:
    #links
    if box.find("h2", class_ = "crayons-story__title") is not None:
        link = box.find("h2", class_ = "crayons-story__title").a
        link = link["href"]
        links.append(link.strip())
    else:
        links.append("None")
        
    #titles
    if box.find("h2", class_ = "crayons-story__title") is not None:
        title = box.find("h2", class_ = "crayons-story__title")
        titles.append(title.text.replace("\n", "").strip())
    else:
        titles.append("None")
        
    #time_uploaded
    if box.find("time", attrs = {"datetime": True}) is not None:
        time_upload = box.find("time", attrs = {"datetime": True})
        time_upload = time_upload["datetime"]
        time_uploaded.append(time_upload)
    else:
        time_uploaded.append("None")
        
    #authors
    if box.find("a", class_ = "crayons-story__secondary fw-medium m:hidden") is not None:
        author = box.find("a", class_ = "crayons-story__secondary fw-medium m:hidden")
        authors.append(author.text.replace("\n", "").strip())
    else:
        authors.append("None")
        
    #tags
    if box.find("div", class_ = "crayons-story__tags") is not None:
        tag = box.find("div", class_ = "crayons-story__tags")
        tags.append(tag.text.replace("\n", " ").strip())
    else:
        tags.append("None")
        
    #reading_times
    if box.find("div",class_ = "crayons-story__save") is not None:
        reading_time = box.find("div",class_ = "crayons-story__save")
        reading_times.append(reading_time.text.replace("\n", "").strip())
    else:
        reading_times.append("None")

In [7]:
blog_df = pd.DataFrame(
    {
        "Link": links,
        "Title": titles,
        "Time_Uploaded": time_uploaded,
        "Author": authors,
        "Tag": tags,
        "Reading_Time": reading_times
    }
)

blog_df = blog_df[blog_df["Link"] != "None"]

In [8]:
blog_df.Link.to_list()


['https://dev.to/analaura/quando-foi-que-a-gente-parou-de-fazer-perguntas-45hc',
 'https://dev.to/neuml/medical-rag-research-with-txtai-2b35',
 'https://dev.to/learnspringtools/springbootfirststepsp1-56na',
 'https://dev.to/learnspringtools/springbootfirststepsp2-4o52',
 'https://dev.to/learnspringtools/springbootfirststepslib-54ki',
 'https://dev.to/vaib/declarative-programming-sql-html-css-prolog-guide-nd0',
 'https://dev.to/nabbisen/uiux-da-gui-mo-na-css-henodui-ying-3563',
 'https://dev.to/mia_koring_962598adda4663/text-compressing-introduction-huffman-coding-in-swift-1o97',
 'https://dev.to/fangmbeng/cyberguard-ai-107i',
 'https://dev.to/sigurdurb/build-a-sanctions-check-in-10-minutes-with-sanctionsnap-api-4moh',
 'https://dev.to/teruxz/devops-kak-po-uchiebniku-vozmozhno-li-ru-3jnb',
 'https://dev.to/huetteldorf/announcing-pyseoa-ts-v020-more-power-more-control-49ji',
 'https://dev.to/codanyks/stop-duct-taping-context-into-prompts-meet-the-mcp-server-hi5',
 'https://dev.to/motchan

In [9]:
article = []
article_link = []

def get_full_content(url2):
    ua = UserAgent()
    userAgent = ua.random
    headers = {"user-agent": userAgent}
    page = requests.get(url2, headers = headers)

    soup2 = BeautifulSoup(page.content, "html.parser")
    #print(url2)



    content = soup2.find("div", class_ = "crayons-article__main")

    paragraphs = content.find_all("p")

    contents = []

    for x in paragraphs:
        contents.append(x.text.replace("\n", " "))

    full_content = " ".join(contents)
    article.append(full_content)
    article_link.append(url2)
    
for i in blog_df.Link:
    get_full_content(i)

article_df = pd.DataFrame(
    {
        "Link": article_link,
        "Article_Content": article
    }
)

In [10]:
merged_df = blog_df.merge(article_df, on = "Link", how = "inner")

In [11]:
pip install langid pycountry nltk

     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
     ---------- ----------------------------- 0.5/1.9 MB 882.6 kB/s eta 0:00:02
     ---------- ----------------------------- 0.5/1.9 MB 882.6 kB/s eta 0:00:02
     ---------------- ----------------------- 0.8/1.9 MB 882.6 kB/s eta 0:00:02
     ---------------- ----------------------- 0.8/1.9 MB 882.6 kB/s eta 0:00:02
     --------------------- ------------------ 1.0/1.9 MB 774.0 kB/s eta 0:00:02
     --------------------- ------------------ 1.0/1.9 MB 774.0 kB/s eta 0:00:02
     --------------------- ------------------ 1.0/1.9 MB 774.0 kB/s eta 0:00:02
     --------------------- ------------------ 1.0/1.9 MB 774.0 kB/s eta 0:00:02
     --------------------- ------------------ 1.0/1.9 MB 774.0 kB/s eta 0:00

In [12]:
from nltk.corpus import stopwords
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the stopwords dataset
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("vader_lexicon")
nltk.download("punkt_tab")


def count_words_without_stopwords(text):
    if isinstance(text, (str, bytes)):
        words = nltk.word_tokenize(str(text))
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return len(filtered_words)
    else:
        0
        
merged_df['Word_Count'] = merged_df["Article_Content"].apply(count_words_without_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
sent = SentimentIntensityAnalyzer()

def get_sentiment(record):
    sentiment_scores = sent.polarity_scores(record)
    compound_score = sentiment_scores['compound']
    
    if compound_score >= 0.05:
        sentiment = 'Positive'
    elif compound_score <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'
        
    return compound_score, sentiment

merged_df[['Compound_Score' ,'Sentiment']] = merged_df['Article_Content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))

In [14]:
import langid
import pycountry

def detect_language(text):
    # Convert Nan to an empty string
    text = str(text) if pd.notna(text) else ''
    
    # Use langid to detect the language
    lang, confidence = langid.classify(text)
    return lang

merged_df['Language'] = merged_df['Article_Content'].apply(detect_language)
merged_df['Language'] = merged_df['Language'].map(lambda code: pycountry.languages.get(alpha_2 = code).name if pycountry.languages.get(alpha_2 = code) else code)

In [15]:
filtered_df = merged_df[merged_df['Language'] == 'English'].reset_index(drop = True)
filtered_df['Reading_Time'] = filtered_df['Reading_Time'].str.replace(' min read', '', regex=False).str.strip().astype(int)
filtered_df.head()

,Link,Title,Time_Uploaded,Author,Tag,Reading_Time,Article_Content,Word_Count,Compound_Score,Sentiment,Language
0,https://dev.to/neuml/medical-rag-research-with...,Medical RAG Research with txtai,2025-06-23T21:23:43Z,David Mezzetti,#ai #llm #rag #vectordatabase,7,txtai is an all-in-one AI framework for semant...,458,0.9858,Positive,English
1,https://dev.to/learnspringtools/springbootfirs...,springboot.firststeps(P1),2025-06-23T21:06:55Z,olver team,,3,personaController.java @DateTimeFormat(patte...,22,0.0000,Neutral,English
2,https://dev.to/vaib/declarative-programming-sq...,"Declarative Programming: SQL, HTML, CSS, Prolo...",2025-06-23T21:02:36Z,Vaiber,#programming #declarative #sql #prolog,7,"Welcome, fellow developers and curious minds! ...",666,0.9989,Positive,English
3,https://dev.to/mia_koring_962598adda4663/text-...,Text Compressing Introduction - Huffman Coding...,2025-06-23T20:49:39Z,Mia Koring,#computerscience #algorithms #swift #beginners,1,Ever wondered how file compression actually wo...,111,0.9603,Positive,English
4,https://dev.to/fangmbeng/cyberguard-ai-107i,CyberGuard AI,2025-06-23T20:46:48Z,Brandon Atonte,,4,The cybersecurity landscape is facing an unpre...,717,-0.9781,Negative,English


In [16]:
# CREATE TABLE IF NOT EXISTS articles(
# Link TEXT,
# Title TEXT,
# Time_Uploaded TIMESTAMP,
# Author TEXT,
# Tag TEXT,
# Reading_Time INTEGER,
# Article_Content TEXT,
# Word_Count INTEGER,
# Compound_Score NUMERIC,
# Sentiment TEXT,
# Language TEXT
# );

!pip install psycopg2

In [ ]:
import psycopg2

db_params = {
    "dbname": "postgres",
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": "5432"
}

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    # SQL Insert Query
    insert_query = """
    INSERT INTO articles (Link, Title, Time_Uploaded, Author, Tag, Reading_Time, Article_Content, Word_Count, Compound_Score, Sentiment, Language)
    VALUES (%s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s)
    ON CONFLICT (Link) DO NOTHING;  -- Avoids duplicate primary key errors
    """
    
    # Insert DataFrame records one by one
    for _, row in filtered_df.iterrows():
        cursor.execute(insert_query, (
            row['Link'], row['Title'], row['Time_Uploaded'],  row['Author'], row['Tag'], row['Reading_Time'],
            row['Article_Content'],row['Word_Count'],row['Compound_Score'],row['Sentiment'],row['Language']
        ))

    # Commit and close
    conn.commit()
    print("Data inserted successfully!")

except Exception as e:
    print(e)

finally:
    if conn:
        cursor.close()
        conn.close

Data inserted successfully!
